### Paxillin movie segmentation and object feature analysis

with Shailaja's movies

Liya Ding, 2024.03

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import umap
from joblib import dump, load
import os.path

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.transforms import Bbox

import tifffile 
import czifile

import skimage
from skimage.morphology import remove_small_objects
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb

from scipy import ndimage
from scipy.ndimage import distance_transform_cdt
from scipy.ndimage import gaussian_filter

from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# function from aicssegmentation tool
from utils.vessel_2d import filament_2d_wrapper
from utils.pre_processing_utils import intensity_normalization


In [ ]:
czimovie_dir =  '/mnt/d/lding/FA/data/FA_movies_SS'
csv_output_dir = '/mnt/d/lding/FA/analysis_results/movies_SS/csv'
plot_output_dir = '/mnt/d/lding/FA/analysis_results/movies_SS/FA_plot'
seg_output_dir = '/mnt/d/lding/FA/analysis_results/movies_SS/FA_seg'

if not os.path.isdir(csv_output_dir):
    os.makedirs(csv_output_dir)
if not os.path.isdir(plot_output_dir):
    os.makedirs(plot_output_dir)
if not os.path.isdir(seg_output_dir):
    os.makedirs(seg_output_dir)

In [ ]:
# build the colormap with iterative tab20
tab20 = cm.get_cmap('tab20', 20)
for i in range(10):
    tab20.colors = np.concatenate([tab20.colors,tab20.colors],axis=0)    
tab20.colors = np.concatenate([np.zeros([1,4]),tab20.colors],axis=0)
# print(tab20.colors)

### for each movie run through all the time points

In [ ]:
# flag for some optinal parts
flag_plot = True
flag_plot_save = True
flag_seg_save = True
flag_csv_save = True
flag_run_all = True
flag_panel_plot = True

In [ ]:
prop_df_pax_all=pd.read_csv(os.path.join('/mnt/d/lding/FA/analysis_results/movies_SS/','cluster_labels_allcells.csv'))

        

In [37]:
newmap = cm.get_cmap('tab20', 1000+1)
newmap.colors = tab20.colors[0:1000+1,:]
newmap.colors[0,:]  = np.zeros([1,4])
newmap.colors[0,3] = 1

In [38]:
newmap.colors.shape

(1001, 4)

In [39]:
filenames = [x for x in os.listdir(czimovie_dir) if os.path.isfile(os.path.join(czimovie_dir, x)) and ('.czi' in x)]

# if not in run all mode, only test with the first image of the first movie
if not flag_run_all:
    filenames = filenames[0:2]

for filenameID in range(len(filenames)):
    filename = filenames[filenameID]
    img = tifffile.imread(os.path.join(czimovie_dir,filename))
    movie_cluster_plot_output_dir = os.path.join(plot_output_dir,  'cluster_plot'+str(filenameID))
    if not os.path.isdir(movie_cluster_plot_output_dir):   os.makedirs(movie_cluster_plot_output_dir)
    
    ### for each movie, make sub-folders for different output for easier viewing in imagej
    movie_label_output_dir = os.path.join(seg_output_dir,  'label'+str(filenameID))
    
    # get the length is the movie
    time_length = img.shape[0]

    # if not in run all mode, only test with the first image of the first movie
    if not flag_run_all:
        time_length = 2

    for timeID in range(time_length):
        input_pax_img = img[timeID,:,:]      
        label_pax_seg = tifffile.imread(os.path.join(movie_label_output_dir, 'SS_pax_seglabel_'+str(filenameID)+'_t'+str(timeID)+'.tif'))   
        max_label = label_pax_seg.max()+1

       
        # print(newmap.colors[:,0:3])
        less_col_prop_df_pax_all = prop_df_pax_all[['filename','cell_ID','time_point','label','cluster_ID_MeanShif']]
        cluster_label_pax_seg = np.zeros_like(label_pax_seg)
        this_cell_table = less_col_prop_df_pax_all[less_col_prop_df_pax_all['cell_ID'] == filenameID]
        this_cell_time = this_cell_table[this_cell_table['time_point'] == timeID]
        # print(filename)
        # print(this_cell_time.iloc[0,0])
        
        for iL in range(1,label_pax_seg.max()+1):  
            # print(iL)
            cluster_ID = this_cell_time[this_cell_time['label']==iL]
            if(cluster_ID.shape[0]==1):
                cluster_label_pax_seg[label_pax_seg==iL] = cluster_ID.iloc[0,-1]+1

        if flag_plot:

            if flag_panel_plot:
                # plot the main outputs together for quick viewing
                fig, ax = plt.subplots(1,3, figsize=(16,8), dpi=256, facecolor='w', edgecolor='k')
                ax[0].imshow(input_pax_img, cmap=plt.cm.gray)
                ax[0].axis('off')

                ax[1].imshow(label_pax_seg, cmap=newmap, vmax = 1001,vmin = 0, interpolation='none')
                ax[1].axis('off')

                ax[2].imshow(cluster_label_pax_seg, cmap=newmap,vmax = 1001,vmin = 0, interpolation='none')
                ax[2].axis('off')
              
                if flag_plot_save: 
                    # save the plots and subplots for easier viewing
                    plt.savefig(os.path.join(movie_cluster_plot_output_dir,'clusters_'+str(filenameID)+'_t'+str(timeID)+'.png'))
                
                if flag_run_all:
                    plt.close(fig) 
                        

                

In [34]:
newmap.colors

array([[0.        , 0.        , 0.        , 1.        ],
       [0.12156863, 0.46666667, 0.70588235, 1.        ],
       [0.68235294, 0.78039216, 0.90980392, 1.        ],
       [1.        , 0.49803922, 0.05490196, 1.        ],
       [1.        , 0.73333333, 0.47058824, 1.        ],
       [0.17254902, 0.62745098, 0.17254902, 1.        ],
       [0.59607843, 0.8745098 , 0.54117647, 1.        ],
       [0.83921569, 0.15294118, 0.15686275, 1.        ],
       [1.        , 0.59607843, 0.58823529, 1.        ],
       [0.58039216, 0.40392157, 0.74117647, 1.        ],
       [0.77254902, 0.69019608, 0.83529412, 1.        ],
       [0.54901961, 0.3372549 , 0.29411765, 1.        ],
       [0.76862745, 0.61176471, 0.58039216, 1.        ],
       [0.89019608, 0.46666667, 0.76078431, 1.        ],
       [0.96862745, 0.71372549, 0.82352941, 1.        ],
       [0.49803922, 0.49803922, 0.49803922, 1.        ],
       [0.78039216, 0.78039216, 0.78039216, 1.        ],
       [0.7372549 , 0.74117647,

In [33]:
newmap.to_numpy

AttributeError: 'ListedColormap' object has no attribute 'to_numpy'

In [ ]:
timeID

In [ ]:
cluster_ID = this_cell_time[this_cell_time['label']==iL]


In [ ]:
iL

In [ ]:
this_cell_time

In [ ]:
cluster_ID.shape[0]


In [ ]:
iL

In [ ]:
this_cell_table['label']

In [ ]:
cluster_ID['cluster_ID_MeanShif']

In [ ]:
label_pax_seg.max()

In [ ]:
'd='+f'{cell_edge_dist:.0f}'+',oo='+f'{regionprops_pax[iL].orientation:.1f}'+',co='+f'{cell_edge_orient:.1f}'+',diff='+f'{diff_orient:.1f}'

In [ ]:
f'{cell_edge_dist:.0f}'

### belows are for debugging, keep it here until no longer needed

In [ ]:
pax_image_label_overlay = label2rgb(label_pax_seg, image=smooth_pax_img, bg_label=0,kind='overlay',alpha=0.5)
plt.rcParams["figure.figsize"] = (15,15)
plt.imshow(pax_image_label_overlay)

In [ ]:
plt.rcParams["figure.figsize"] = (15,15)
plt.imshow(pax_image_label_overlay, cmap=plt.cm.RdBu_r,vmax=0.1,vmin=0)
# plt.quiver(obj_Y,obj_X, cell_U,cell_V,color='blue')
Q = plt.quiver(obj_Y,obj_X, -obj_U,obj_V,color='m')#width=0.005,headwidth=0.1,headaxislength=0.02,headlength=0.01)
# plt.quiver(obj_Y,obj_X, cell_U,cell_V,color='blue')

plt.axis('off')

In [ ]:
plt.imshow(distance_taxicab)

grid_mask = np.zeros_like(new_cell_mask)
grid_mask[::30,::30] = new_cell_mask[::30,::30]
to_plot_X = X[grid_mask>0]
to_plot_Y = Y[grid_mask>0]
to_plot_U = n_v[grid_mask>0]
to_plot_H = n_h[grid_mask>0]

# plt.contour(X, Y, distance_taxicab,6,linewidths=2)    
plt.quiver(to_plot_X,to_plot_Y, -to_plot_U,to_plot_H,color='blue')
plt.axis('off')